In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


nursing_houses  = [
    {"nh_id": "00001", "name": "บ้านพักสุขใจ", "province": "กรุงเทพฯ", "address": "123 ถนนสุขใจ", "price": 15000, "map": "https://goo.gl/maps/xyz1", "phone_number": "081-2345678", "site": "https://sukhjaihome.com", "Date": "2024-01-10", "status": "Active"},
    {"nh_id": "00002", "name": "บ้านพักอุ่นใจ", "province": "เชียงใหม่", "address": "456 ถนนอบอุ่น", "price": 20000, "map": "https://goo.gl/maps/xyz2", "phone_number": "082-3456789", "site": "https://warmhome.com", "Date": "2024-01-11", "status": "Active"},
    {"nh_id": "00003", "name": "บ้านพักร่มเย็น", "province": "ภูเก็ต", "address": "789 ถนนสงบสุข", "price": 160000, "map": "https://goo.gl/maps/xyz3", "phone_number": "083-4567890", "site": "https://coolhome.com", "Date": "2024-01-12", "status": "Active"},
    {"nh_id": "00004", "name": "บ้านพักอุ่นใจมากๆ", "province": "เชียงราย", "address": "12345 ถนนสุขใจมากๆ", "price": 12000, "map": "https://goo.gl/maps/xyz212", "phone_number": "082-3456789", "site": "https://warmhome.com", "Date": "2024-01-11", "status": "Active"},
]


# แนะนำบ้านที่ใกล้เคียงที่ User คลิ้กอ่าน

# TF-IDF

In [ ]:

df = pd.DataFrame(nursing_houses)


vectorizer = TfidfVectorizer()
features = df['province'] + " " + df['address']  # รวม province และ address
X = vectorizer.fit_transform(features)


print("TF-IDF Matrix:")
print(X.toarray())

TF-IDF Matrix:
[[0.46516193 0.         0.         0.         0.46516193 0.46516193
  0.         0.46516193 0.36673901 0.         0.         0.
  0.         0.         0.        ]
 [0.         0.         0.52547275 0.         0.         0.
  0.         0.         0.         0.         0.52547275 0.
  0.52547275 0.         0.41428875]
 [0.         0.         0.         0.57735027 0.         0.
  0.         0.         0.         0.57735027 0.         0.
  0.         0.57735027 0.        ]
 [0.         0.48546061 0.         0.         0.         0.
  0.48546061 0.         0.38274272 0.         0.         0.48546061
  0.         0.         0.38274272]]


In [ ]:

cosine_sim = cosine_similarity(X, X)

In [ ]:

def get_similar_nursing_homes(nh_id, df, cosine_sim):
    idx = df.index[df['nh_id'] == nh_id].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:2]  # เอาผลลัพธ์ที่คล้ายกันมากที่สุด 1 อันดับ
    similar_indices = [i[0] for i in sim_scores]
    return df.iloc[similar_indices]


similar_homes = get_similar_nursing_homes("00002", df, cosine_sim)
print("บ้านพักที่คล้ายกับ 00002:")
print(similar_homes[['nh_id', 'name', 'province', 'address']])

บ้านพักที่คล้ายกับ 00002:
   nh_id               name  province             address
3  00004  บ้านพักอุ่นใจมากๆ  เชียงราย  12345 ถนนสุขใจมากๆ


# Sentence Transformer

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')


def encode_texts(texts):
    return model.encode(texts)

# รวมข้อความที่ต้องการแปลง
texts = df['name'] + " " + df['address']


text_embeddings = encode_texts(texts.tolist())


cosine_sim = cosine_similarity(text_embeddings)


cosine_sim_df = pd.DataFrame(cosine_sim, columns=df['name'], index=df['name'])


cosine_sim_df

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

name,บ้านพักสุขใจ,บ้านพักอุ่นใจ,บ้านพักร่มเย็น,บ้านพักอุ่นใจมากๆ
name,,,,
บ้านพักสุขใจ,1.000000,0.736334,0.753833,0.911510
บ้านพักอุ่นใจ,0.736334,1.000000,0.702184,0.759641
บ้านพักร่มเย็น,0.753833,0.702184,1.000000,0.686920
บ้านพักอุ่นใจมากๆ,0.911510,0.759641,0.686920,1.000000


In [ ]:
def get_similar_nursing_homes_by_id(nh_id, df, cosine_sim_df, top_n=1):

    if nh_id not in df['nh_id'].values:
        raise ValueError(f"ID บ้านพัก '{nh_id}' ไม่พบในข้อมูล")


    nh_name = df.loc[df['nh_id'] == nh_id, 'name'].values[0]


    sim_scores = cosine_sim_df[nh_name].sort_values(ascending=False)

    # เอาเฉพาะบ้านพักที่ไม่ใช่ตัวเลือกที่ผู้ใช้เลือก
    sim_scores = sim_scores[sim_scores.index != nh_name]


    similar_nh = sim_scores.head(top_n)


    return similar_nh.index.tolist(), similar_nh.values.tolist()


similar_nh_names, similar_nh_scores = get_similar_nursing_homes_by_id('00001', df, cosine_sim_df)


print("บ้านพักที่คล้ายคลึงกันมากที่สุด:")
for name, score in zip(similar_nh_names, similar_nh_scores):
    print(f"บ้านพัก: {name}, Cosine Similarity: {score:.4f}")


บ้านพักที่คล้ายคลึงกันมากที่สุด:
บ้านพัก: บ้านพักอุ่นใจมากๆ, Cosine Similarity: 0.9115


# แนะนำตามข้อมูลของแผนการเกษียณ user

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

retirement_plans = [
    {
        "financial_id": "RP001",
        "planName": "Plan A",
        "birth_date": "1965-08-25",
        "age": 59,
        "retirement_age": 60,
        "expect_lifespan": 85,
        "current_savings": 500000,
        "current_savings_returns": 4.0,
        "monthly_income": 50000,
        "monthly_expenses": 30000,
        "current_total_investment": 200000,
        "investment_return": 6.0,
        "expected_monthly_expenses": 25000,
        "expected_inflation": 3.0,
        "annual_expense_increase": 2.5,
        "annual_savings_return": 4.5,
        "annual_investment_return": 5.5,
        "user_id": "user001"
    },
    {
        "financial_id": "RP002",
        "planName": "Plan B",
        "birth_date": "1970-02-10",
        "age": 54,
        "retirement_age": 65,
        "expect_lifespan": 90,
        "current_savings": 700000,
        "current_savings_returns": 5.0,
        "monthly_income": 60000,
        "monthly_expenses": 35000,
        "current_total_investment": 300000,
        "investment_return": 7.0,
        "expected_monthly_expenses": 30000,
        "expected_inflation": 3.5,
        "annual_expense_increase": 3.0,
        "annual_savings_return": 5.5,
        "annual_investment_return": 6.5,
        "user_id": "user002"
    }
]

In [ ]:
df_retirement = pd.DataFrame(retirement_plans)
df_nursing = pd.DataFrame(nursing_houses)


retirement_descriptions = df_retirement['planName'] + " - " + df_retirement['user_id']
nursing_descriptions = df_nursing['name'] + " - " + df_nursing['province']


retirement_embeddings = model.encode(retirement_descriptions.tolist())
nursing_embeddings = model.encode(nursing_descriptions.tolist())

In [ ]:
def recommend_nursing_home(user_id, retirement_embeddings, nursing_embeddings, df_retirement, df_nursing, top_n=2):

    user_plan = df_retirement[df_retirement['user_id'] == user_id].iloc[0]

    # คำนวณงบประมาณที่สามารถจ่ายได้
    budget = user_plan['monthly_income'] - user_plan['monthly_expenses']


    similarity_scores = []
    for nh_embedding in nursing_embeddings:

        nh_embedding = np.array(nh_embedding)
        similarity_scores.append(np.dot(retirement_embeddings[0], nh_embedding) /
                                 (np.linalg.norm(retirement_embeddings[0]) * np.linalg.norm(nh_embedding)))


    similarity_scores = np.array(similarity_scores)


    affordable_nursing_homes = df_nursing[df_nursing['price'] <= budget]


    if not affordable_nursing_homes.empty:
        # แนะนำบ้านพักที่มีราคาต่ำกว่าหรือเท่ากับงบประมาณ และเรียงตามความคล้ายคลึง
        affordable_indices = affordable_nursing_homes.index
        affordable_similarity_scores = similarity_scores[affordable_indices]
        recommended_indices = affordable_indices[np.argsort(affordable_similarity_scores)[-top_n:][::-1]]
    else:
        recommended_indices = np.argsort(similarity_scores)[-top_n:][::-1]

    recommended_nursing_homes = df_nursing.iloc[recommended_indices]

    return recommended_nursing_homes[['name', 'province', 'price']]


recommended_homes = recommend_nursing_home('user001', retirement_embeddings, nursing_embeddings, df_retirement, df_nursing)


print("บ้านพักที่แนะนำสำหรับ user001:")
print(recommended_homes)

บ้านพักที่แนะนำสำหรับ user001:
                name   province  price
3  บ้านพักอุ่นใจมากๆ   เชียงราย  12000
1      บ้านพักอุ่นใจ  เชียงใหม่  20000


# Hybrid TF-IDF รวมกับ Feature Vector

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


nursing_houses = [
    {"nh_id": "00001", "name": "บ้านพักสุขใจ", "province": "กรุงเทพฯ", "address": "123 ถนนสุขใจ", "price": 15000, "map": "https://goo.gl/maps/xyz1", "phone_number": "081-2345678", "site": "https://sukhjaihome.com", "Date": "2024-01-10", "status": "Active"},
    {"nh_id": "00002", "name": "บ้านพักอุ่นใจ", "province": "เชียงใหม่", "address": "456 ถนนอบอุ่น", "price": 20000, "map": "https://goo.gl/maps/xyz2", "phone_number": "082-3456789", "site": "https://warmhome.com", "Date": "2024-01-11", "status": "Active"},
    {"nh_id": "00003", "name": "บ้านพักร่มเย็น", "province": "ภูเก็ต", "address": "789 ถนนสงบสุข", "price": 160000, "map": "https://goo.gl/maps/xyz3", "phone_number": "083-4567890", "site": "https://coolhome.com", "Date": "2024-01-12", "status": "Active"},
    {"nh_id": "00004", "name": "บ้านพักอุ่นใจมากๆ", "province": "เชียงราย", "address": "12345 ถนนสุขใจมากๆ", "price": 12000, "map": "https://goo.gl/maps/xyz212", "phone_number": "082-3456789", "site": "https://warmhome.com", "Date": "2024-01-11", "status": "Active"},
]

retirement_plans = [
    {
        "financial_id": "RP001",
        "planName": "Plan A",
        "monthly_income": 50000,
        "monthly_expenses": 30000,
        "expected_monthly_expenses": 25000,
        "user_id": "user001"
    },
    {
        "financial_id": "RP002",
        "planName": "Plan B",
        "monthly_income": 60000,
        "monthly_expenses": 35000,
        "expected_monthly_expenses": 30000,
        "user_id": "user002"
    }
]



def create_tfidf_embeddings(nursing_houses):
    descriptions = [
        f"{nh['name']} {nh['province']} {nh['address']}" for nh in nursing_houses
    ]
    vectorizer = TfidfVectorizer()
    tfidf_embeddings = vectorizer.fit_transform(descriptions).toarray()
    return tfidf_embeddings


def create_feature_embeddings(nursing_houses):
    prices = [nh["price"] for nh in nursing_houses]
    scaler = MinMaxScaler()
    price_embeddings = scaler.fit_transform(np.array(prices).reshape(-1, 1))
    return price_embeddings


def create_hybrid_embeddings(nursing_houses):
    tfidf_embeddings = create_tfidf_embeddings(nursing_houses)
    feature_embeddings = create_feature_embeddings(nursing_houses)
    hybrid_embeddings = np.hstack([tfidf_embeddings, feature_embeddings])
    return hybrid_embeddings


house_embeddings = create_hybrid_embeddings(nursing_houses)


print("House Embeddings Shape:", house_embeddings.shape)
print("Sample House Embedding:", house_embeddings[0])


House Embeddings Shape: (4, 22)
Sample House Embedding: [0.34268479 0.         0.         0.         0.27017662 0.34268479
 0.         0.68536957 0.         0.34268479 0.27017662 0.
 0.         0.         0.         0.         0.         0.
 0.17882714 0.         0.         0.02027027]


In [ ]:

def calculate_cosine_similarity(house_embeddings):
    return cosine_similarity(house_embeddings)


def recommend_nursing_houses(user_id, nursing_houses, retirement_plans, top_n=3):


    user_plan = next((plan for plan in retirement_plans if plan["user_id"] == user_id), None)
    if not user_plan:
        return f"ไม่พบแผนเกษียณสำหรับผู้ใช้ {user_id}"


    budget = user_plan["monthly_income"] - user_plan["monthly_expenses"]


    house_embeddings = create_hybrid_embeddings(nursing_houses)


    #สร้างตาราง เรียงลำดับบ้าน
    similarity_matrix = calculate_cosine_similarity(house_embeddings)

    #บ้านต้นแบบ ของจริงอาจดึงจากประวัติการกดดูของ user
    query_index = 0
    similarity_scores = similarity_matrix[query_index]


    affordable_indices = [i for i, house in enumerate(nursing_houses) if house["price"] <= budget]

    if not affordable_indices:
        return f"ไม่มีบ้านพักที่อยู่ในงบประมาณของคุณ ({budget} บาท/เดือน)"


    filtered_houses = [(i, similarity_scores[i]) for i in affordable_indices]
    filtered_houses = sorted(filtered_houses, key=lambda x: x[1], reverse=True)[:top_n]


    recommendations = [
        {
            "nh_id": nursing_houses[i]["nh_id"],
            "name": nursing_houses[i]["name"],
            "province": nursing_houses[i]["province"],
            "price": nursing_houses[i]["price"],
            "similarity": round(score, 4)
        }
        for i, score in filtered_houses
    ]

    return recommendations


user_id = "user001"
recommendations = recommend_nursing_houses(user_id, nursing_houses, retirement_plans, top_n=1)


print("บ้านพักแนะนำสำหรับ", user_id)
for rec in recommendations:
    print(rec)

บ้านพักแนะนำสำหรับ user001
{'nh_id': '00001', 'name': 'บ้านพักสุขใจ', 'province': 'กรุงเทพฯ', 'price': 15000, 'similarity': 1.0}


# recommendation with LLM

In [ ]:
!pip install langchain langchain-google-genai
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 562.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.7 MB/s eta 0:00:00


## Gemini

In [ ]:
import json
nursing_houses_str = json.dumps(nursing_houses, ensure_ascii=False, indent=2).replace("{", "{{").replace("}", "}}")
retirement_plans_str = json.dumps(retirement_plans, ensure_ascii=False, indent=2).replace("{", "{{").replace("}", "}}")

example_json = json.dumps(nursing_houses[0], ensure_ascii=False, indent=2).replace("{", "{{").replace("}", "}}")


example_json

'{{\n  "nh_id": "00001",\n  "name": "บ้านพักสุขใจ",\n  "province": "กรุงเทพฯ",\n  "address": "123 ถนนสุขใจ",\n  "price": 15000,\n  "map": "https://goo.gl/maps/xyz1",\n  "phone_number": "081-2345678",\n  "site": "https://sukhjaihome.com",\n  "Date": "2024-01-10",\n  "status": "Active"\n}}'

In [ ]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate


GOOGLE_API_KEY = 


system_prompt = f"""You are an AI that is good at recommending homes. You will give short answers in a single element structured JSON in nursing_house.
Most importantly, you will not respond to anything else except JSON. The format of the JSON must strictly match the following structure
{example_json}
Here is the information you need to consider: Home data in the database: {nursing_houses_str}
This is the user's retirement plan data. It will contain important financial information.
{retirement_plans_str}"""
# user_prompt = "recommending random houses name base on database"
user_prompt = "Help recommend the most expensive and luxurious houses in the database."

llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", user_prompt)
])

response = llm.invoke(prompt.format())
print(response)


{
  "nh_id": "00003",
  "name": "บ้านพักร่มเย็น",
  "province": "ภูเก็ต",
  "address": "789 ถนนสงบสุข",
  "price": 160000,
  "map": "https://goo.gl/maps/xyz3",
  "phone_number": "083-4567890",
  "site": "https://coolhome.com",
  "Date": "2024-01-12",
  "status": "Active"
}


## llama

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from huggingface_hub import login

login("token_here")

### เซฟโมเดล

In [ ]:
from transformers import pipeline


llm_pipeline = pipeline(
    "text-generation",
    model="meta-llama/Llama-3.2-1B-Instruct",
    max_new_tokens=256,
    temperature=0.7,
    repetition_penalty=1.1
)

save_path = "/content/drive/My Drive/saved_model"
llm_pipeline.model.save_pretrained(save_path)
llm_pipeline.tokenizer.save_pretrained(save_path)

print(f"Model saved to {save_path}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


Model saved to /content/drive/My Drive/saved_model


In [ ]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from langchain.llms import HuggingFacePipeline
from langchain_core.prompts import ChatPromptTemplate


load_path = "/content/drive/My Drive/saved_model"
model = AutoModelForCausalLM.from_pretrained(load_path)
tokenizer = AutoTokenizer.from_pretrained(load_path)

# สร้าง pipeline ใหม่จากโมเดลที่โหลดมา
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.7,
    repetition_penalty=1.1
)


llm = HuggingFacePipeline(pipeline=llm_pipeline)

Device set to use cuda:0
<ipython-input-29-cacf6002a0ca>:21: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


In [ ]:
# from langchain.llms import HuggingFacePipeline
# from transformers import pipeline
# from langchain_core.prompts import ChatPromptTemplate

# # llm_pipeline = pipeline("text-generation", model="meta-llama/Llama-3.2-1B")

# llm_pipeline = pipeline(
#     "text-generation",
#     # model="meta-llama/Llama-3.2-1B",
#     model="meta-llama/Llama-3.2-1B-Instruct",
#     max_new_tokens=256,  # เพิ่มจำนวนคำตอบให้ยาวขึ้น
#     temperature=0.7,  # ควบคุมความสร้างสรรค์ (ค่ามากขึ้น = สุ่มมากขึ้น)
#     repetition_penalty=1.1  # ป้องกันการวนลูปตอบซ้ำ
# )

# # ใช้งานผ่าน LangChain
# llm = HuggingFacePipeline(pipeline=llm_pipeline)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-4-e4099af056f0>:17: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


In [ ]:
system_prompt = f"""You are an AI that is good at recommending homes. You will give short answers in a single element structured JSON in nursing_house.
Most importantly, you will not respond to anything else except JSON. The format of the JSON must strictly match the following structure
{example_json}
Here is the information you need to consider: Home data in the database: {nursing_houses_str}
This is the user's retirement plan data. It will contain important financial information.
{retirement_plans_str}"""
user_prompt = "Help recommend the most expensive and luxurious houses in the database."


# สร้าง prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", user_prompt)
])


response = llm.invoke(prompt.format())
print(response)

{
  "nh_id": "00003",
  "name": "บ้านพักร่มเย็น",
  "province": "ภูเก็ต",
  "address": "789 ถนนสงบสุข",
  "price": 160000,
  "map": "https://goo.gl/maps/xyz3",
  "phone_number": "083-4567890",
  "site": "https://coolhome.com",
  "Date": "2024-01-12",
  "status": "Active"
}
